<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/CP2_KoSpacing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 띄어쓰기 검사 모델 구현하기

## 필요한 패키지 다운로드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-qfjw6azi
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-qfjw6azi
     |████████████████████████████████| 460.3 MB 7.9 kB/s 
     |████████████████████████████████| 462 kB 60.3 MB/s 
     |████████████████████████████████| 132 kB 57.5 MB/s 
     |████████████████████████████████| 1.2 MB 46.5 MB/s 
     |████████████████████████████████| 14.8 MB 31.4 MB/s 
     |████████████████████████████████| 4.0 MB 26.0 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268580 sha256=e801593666662908c4a8782bd6632408ce8360fe1078268a287b7d9a53e7c6ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-eej18plh/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68724 sha256=5e0ef85793cb5aa3e684df60c4b913f523cfe8aa900f8c37786b50e215c73d74
  Stored in 

In [3]:
pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-_hsl3e5y
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-_hsl3e5y
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=a67af76d19f68d884b434116366c59b54fcd50e983d4b60bb2333362dc1f5b6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-qnj6jo95/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [4]:
pip install soynlp

     |████████████████████████████████| 416 kB 26.7 MB/s 


In [5]:
pip install customized_konlpy

     |████████████████████████████████| 881 kB 28.1 MB/s 
     |████████████████████████████████| 19.4 MB 5.5 MB/s 
     |████████████████████████████████| 448 kB 54.1 MB/s 


In [6]:
import os
from hanspell import spell_checker

import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *

## 데이터 불러오기

In [7]:
root = '/content/drive/MyDrive/news_class9x1400/'
categories = os.listdir(root)

dataset = []

for cat in categories:
  files = os.listdir(root + cat)
  for i, f in enumerate(files):
    fname = root + cat + '/' + f
    file = open(fname, 'r', encoding='utf-8')
    strings = file.read()
    dataset.append([strings])
    file.close()

print(len(dataset))

12600


In [8]:
dataset[1427]

['한 달 새 외환보유고 48억 달러 큰폭 증가, 왜?\n지난 달 국내 외환보유액이 크게 늘었다.\n2일 한국은행이 내놓은 자료를 보면, 지난 3월 말 현재 외환보유액은 2063억 4000만 달러로 집계됐다.\n2월 말 2015억 4000만 달러에 비해서 48억 달러가 증가한 금액이다.\n지난 2006년 11월 48억 달러 증가이후 가장 큰 폭으로 증가했다고 한은쪽은 덧붙였다.\n한은은 이같은 외환보유액 증가 이유에 대해, 지난달 유로화와 엔화 등 다른 나라 통화가 상대적으로 강세를 보였기 때문이라고 설명했다.\n이들 나라의 통화로 표시된 자산의 달러 환산액이 크게 증가했다는 것이다.\n또 지난달에는 국민연금과 통화스와프 만기도래분 5억 달러가 새로 들어왔고, 한은이 가지고 있던 외환의 운영수익도 일부 발생한 것으로 알려졌다.\n하근철 한은 국제기획팀 차장은 지난 달의 경우 (외환) 증가 요인만 있었고, 감소 요인은 없었다면서 은행의 외화자금 조달 사정이 크게 좋아져서 추가로 (한은이) 달러 공급에 나서지 않았다고 말했다.\n외환당국에서 꼽는 달러 감소 요인은 쉽게 말하면, 외환시장에 개입해 달러를 내다 파는 것을 나타내기도 한다.\n이미 작년 한해동안 정부는 인위적으로 외환시장에 사실상 개입하면서 600억 달러 정도의 외환보유고를 날려 버리기도 했다.\n대신 올 들어서면서 정부는 이른바 심리적인 저지선인 외환보유고 2000억 달러를 지키기 위해 시장 개입을 최대한 자제해 왔다.\n대신 통화스와프 협정을 맺은 미국에게 현재 300억 달러 수준의 스와프 규모를 1000억 달러까지 확대해달라고 요청하거나, 일본 등과의 스와프 기간 연장 등을 통해 외환시장 안정에 나서고 있다.\n한편, 3월 말 기준으로 외환보유액 구성은 유가증권이 1842억 4000만 달러(89.3%)로 가장많다.\n이어 예치금 213억 9000만 달러(10.4%), 금 8000만 달러(0.04%) 등의 순이다.']

## 데이터 전처리

In [9]:
!pip install kss

     |████████████████████████████████| 42.4 MB 1.2 MB/s 
     |████████████████████████████████| 175 kB 40.4 MB/s 
  Created wheel for kss: filename=kss-3.4-py3-none-any.whl size=42449209 sha256=ae2d905781ee0c04d3ca45157c78701ae4f8a004da8a2df94afacfa50309fe1d
  Stored in directory: /root/.cache/pip/wheels/06/8e/5b/305f0a804fba3943f353f1b0e3cb1fad39e4f5ae4893ea9590
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=425be92b560c0da99ac14f56c5f248f84b8fc58610c32102676cb80d5a4a9bd5
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built kss emoji


In [11]:
import kss 

process_dataset = []

for sentences in dataset:
  sentences = sentences[0]
  sentences = sentences.replace("'", '')
  sentences = sentences.replace('"', '')
  sentences = sentences.split("\n")

  for sentence in sentences:
    process_dataset.extend(kss.split_sentences(sentence))


len(process_dataset)

[Korean Sentence Splitter]: Initializing Pynori...


KeyboardInterrupt: ignored

In [12]:
# 4h 36m 

len(process_dataset)

27954

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(process_dataset, test_size = 0.2, random_state  = 42)
print(len(train), len(test))

22363 5591


In [14]:
def save_datafile(file_name, dataset):
  with open(
      os.path.join(root, file_name), mode = 'w', encoding='utf-8'
  ) as f:
    for data in dataset:
      f.write(data)

save_datafile('train.txt', train)
save_datafile('test.txt', test)

print('train, test dataset 저장 완료!')

train, test dataset 저장 완료!
